Firstly, we import necessarcy packages, eg. MatCalc, MatGL and Pymatgen.

In [1]:
from __future__ import annotations

import json
import random
import warnings

import matgl
import numpy as np
import pandas as pd
from matcalc.elasticity import ElasticityCalc
from matcalc.phonon import PhononCalc
from matgl.ext.ase import PESCalculator
from pymatgen.core import Structure

# To suppress warnings for clearer output
warnings.simplefilter("ignore")

We leverage the physical constants provided by SciPy to convert stress units, ensuring compatibility with the ASE calculator.

In [2]:
from scipy import constants

eVA3ToGPa = constants.e / (constants.angstrom) ** 3 / constants.giga

Next, I will show you how to run MatCalc-Bench. Here, TensorNet-MatPES-PBE-v2025.1-PES and M3GNet-MatPES-PBE-v2025.1-PES are used to perform elastic modulus calculations. The approach is similar for other models and other benchmarks. For demonstration purposes only, I randomly sample 1% of the entire test dataset.

In [3]:
all_results = {}

for model_name in [
    "M3GNet-MatPES-PBE-v2025.1-PES",
    "TensorNet-MatPES-PBE-v2025.1-PES",
]:
    potential = matgl.load_model(model_name)
    calculator = PESCalculator(potential, stress_weight=1 / eVA3ToGPa)

    with open("MatCalc-Bench_data/elastic.json") as f:
        entries = json.load(f)

    random.seed(2025)
    sampled_entries = random.sample(entries, len(entries) // 100)

    elastc_calc = ElasticityCalc(calculator, fmax=0.05, relax_structure=True)

    bulk_modulus_ae = []
    shear_modulus_ae = []

    for entry in sampled_entries:
        mp_id = entry["mp_id"]
        if mp_id not in all_results:
            all_results[mp_id] = {
                "mp_id": mp_id,
                "formula": entry["formula"],
                "K-DFT": entry["bulk_modulus_vrh"],
                "G-DFT": entry["shear_modulus_vrh"],
                "K-M3GNet-MatPES": None,
                "G-M3GNet-MatPES": None,
                "K-TensorNet-MatPES": None,
                "G-TensorNet-MatPES": None,
            }
        structure = Structure.from_dict(entry["structure"])
        properties = elastc_calc.calc(structure)

        predicted_bulk = properties["bulk_modulus_vrh"] * eVA3ToGPa
        predicted_shear = properties["shear_modulus_vrh"] * eVA3ToGPa

        if model_name.startswith("M3GNet"):
            all_results[mp_id]["K-M3GNet-MatPES"] = predicted_bulk
            all_results[mp_id]["G-M3GNet-MatPES"] = predicted_shear
        elif model_name.startswith("TensorNet"):
            all_results[mp_id]["K-TensorNet-MatPES"] = predicted_bulk
            all_results[mp_id]["G-TensorNet-MatPES"] = predicted_shear

df = pd.DataFrame(list(all_results.values()))

df = df[
    [
        "mp_id",
        "formula",
        "K-DFT",
        "G-DFT",
        "K-M3GNet-MatPES",
        "G-M3GNet-MatPES",
        "K-TensorNet-MatPES",
        "G-TensorNet-MatPES",
    ]
]

df.to_csv("MatCalc-Bench_data/elastic.csv", index=False)

You can certainly perform the full benchmark by slightly modifying the codes provided below, but please be aware that the whole process is considerably time consuming. Therefore, I recommend submitting separate jobs to your cluster for every material—or every few materials, depending on your resource availability and workflow preferences.

Finally, I will demonstrate how to do t-test to rigourously evaluate whether the means between two distributions are equal. For reference, this is an explanation:
https://www.jmp.com/en/statistics-knowledge-portal/t-test/two-sample-t-test

In [4]:
from scipy.stats import ttest_ind, ttest_rel

Read the data obtained from the above workflow. Here I used all the elastic moduli calculated by myself.

In [5]:
df = pd.read_csv("MatCalc-Bench_data/my_elastic.csv")
df = df.dropna()

In [6]:
for col in df.columns:
    if (col.startswith("K") or col.startswith("G")) and not col.endswith("DFT"):
        prop, architecture, dataset = col.split("-")
        df[f"AE {prop}-{architecture}-{dataset}"] = np.abs(df[col] - df[f"{prop}-DFT"])

In [7]:
def analyze_stats(prop):
    data = df[[c for c in df.columns if c.startswith(f"AE {prop}")]]
    means = data.describe().loc["mean"]
    best_model = means.idxmin()
    stats = []
    for col in data.columns:
        _, architecture, dataset = col.split("-")
        result_ind = ttest_ind(data[col], data[best_model], equal_var=False)
        result_rel = ttest_rel(data[col], data[best_model])

        stats.append([dataset, architecture, data[col].mean(), data[col].std(), result_ind.pvalue, result_rel.pvalue])
    stats = pd.DataFrame(
        stats,
        columns=["Dataset", "Architecture", f"{prop} MAE", f"{prop} STDAE", f"{prop} p_diff_ind", f"{prop} p_diff_rel"],
    )
    stats[f"{prop} sig_diff_ind"] = stats[f"{prop} p_diff_ind"] < 0.05
    stats[f"{prop} sig_diff_rel"] = stats[f"{prop} p_diff_rel"] < 0.05
    return stats

In [8]:
k_stats = analyze_stats("K")
g_stats = analyze_stats("G")

In [9]:
index_cols = ["Dataset", "Architecture"]
all_stats = k_stats.set_index(index_cols).join(g_stats.set_index(index_cols), on=index_cols)

In [10]:
all_stats

K MAE    K STDAE  K p_diff_ind  K p_diff_rel  \
Dataset Architecture                                                     
MatPES  M3GNet        26.196201  30.898882  2.330507e-65  5.884562e-93   
        TensorNet     18.109873  21.959349  1.299977e-06  5.826557e-16   
        CHGNet        23.832622  28.142081  4.607903e-45  5.131494e-71   
MPF     M3GNet        21.412454  27.601616  8.678037e-24  1.122304e-40   
        TensorNet     24.485361  34.354847  1.028958e-40  2.535476e-68   
MPtrj   CHGNet        17.504289  59.987498  7.311039e-02  5.175137e-02   
OMat24  TensorNet     15.675647  22.726589  1.000000e+00           NaN   

                      K sig_diff_ind  K sig_diff_rel         G MAE  \
Dataset Architecture                                                 
MatPES  M3GNet                  True            True  2.339403e+08   
        TensorNet               True            True  2.339403e+08   
        CHGNet                  True            True  2.339403e+08   
MPF     M3GNet                  True            True  2.339404e+08   
        TensorNet               True            True  2.339404e+08   
MPtrj   CHGNet                 False           False  2.339403e+08   
OMat24  TensorNet              False           False  2.339403e+08   

                           G STDAE  G p_diff_ind   G p_diff_rel  \
Dataset Architecture                                              
MatPES  M3GNet        1.287635e+10           1.0   3.485811e-03   
        TensorNet     1.287635e+10           1.0            NaN   
        CHGNet        1.287635e+10           1.0   1.296427e-02   
MPF     M3GNet        1.287635e+10           1.0   7.821633e-40   
        TensorNet     1.287635e+10           1.0  8.448405e-100   
MPtrj   CHGNet        1.287635e+10           1.0   1.389405e-93   
OMat24  TensorNet     1.287635e+10           1.0   2.721797e-01   

                      G sig_diff_ind  G sig_diff_rel  
Dataset Architecture                                  
MatPES  M3GNet                 False            True  
        TensorNet              False           False  
        CHGNet                 False            True  
MPF     M3GNet                 False            True  
        TensorNet              False            True  
MPtrj   CHGNet                 False            True  
OMat24  TensorNet              False           False